In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-225560
Azure region: eastus2
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-225560


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FVUQGY6M3 to authenticate.


In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "free-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found CLuster")
except:
    compute_config = AmlCompute.provisioning_configuration(
                vm_size = "Standard_D2_V2",
                max_nodes= 4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

Found CLuster


In [23]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({ "--C" : uniform(0.1,0.9)

})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script = "train.py",
    compute_target = compute_target, 
    environment = sklearn_env,
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, hyperparameter_sampling= ps,
                    primary_metric_name = "Accuracy",
                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                    policy = policy,
                    max_total_runs=20
 )

In [22]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

KeyError: 'log_files'

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()["runDefinition"]["arguments"])
print(best_run.get_file_names())
model = best_run.register_model(model_name = 'best-model-hpd', model_path = "outputs-hpd/model.joblib")
### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
path =  "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
### YOUR CODE HERE ###
ds = Dataset.Tabular.from_delimited_files(path)

# TODO: Split data into train and test sets.

### YOUR CODE HERE ###a


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
X, y = clean_data(ds)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= "classification",
    primary_metric= "accuracy",
    training_data= X_train,
    label_column_name=y_train,
    n_cross_validations=5)

In [2]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=False)
automl_run.wait_for_completion()
### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.
best_run = automl_run.get_best_child()
### YOUR CODE HERE ###
model = best_run.register_model(model_name = 'best-model-hpd', model_path = "outputs-hpd/model.joblib")
### YOUR CODE HERE ###